In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 03
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [2]:
tickers_df = pd.read_csv('Tickers_Example.csv')
#When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
#Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

#Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

# Function produces the daily closing prices of a stock
def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
all_prices = pd.DataFrame(stocks_dict)

tickers_list_temp = tickers_list.copy()
#Some tickers are delisted, so we drop those columns entirely
for i in range (tickers_num):
    if all_prices[tickers_list[i]].isnull().all():
        all_prices.drop(columns=[tickers_list[i]], inplace=True)
        tickers_list_temp.remove(tickers_list[i]) #Getting rid of the tickers from our list too to match

tickers_list = tickers_list_temp.copy()
tickers_num = len(tickers_list)

#Drop the rows at the bottom of the DataFrame where only a few stocks have values and where the dates are strange
all_prices = all_prices.dropna() 

display(tickers_list)

#Display the entire DataFrame, so that we can manually check to make sure the values are sensible
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):

display(all_prices)


- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CSCO',
 'CVS',
 'GM',
 'GOOG',
 'JPM',
 'KMI',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NKE',
 'ORCL',
 'OXY',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SBUX',
 'SLB',
 'SO',
 'SPG',
 'T',
 'TD.TO',
 'TGT',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SO,SPG,T,TD.TO,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-16,124.265907,102.991287,114.960403,257.676666,45.678761,154.593002,140.325378,255.210007,36.480061,259.920013,...,57.139629,107.247574,19.613203,76.842155,174.705963,169.330246,347.556396,205.282028,155.261246,50.686966
2021-03-17,123.464310,97.601997,114.376953,258.410065,46.006893,156.786499,141.676697,263.589996,36.780502,264.149994,...,56.626530,108.208481,19.717846,76.935684,172.951233,171.442841,345.184448,205.863846,153.191956,51.298893
2021-03-18,119.278252,96.421921,114.853439,260.971985,46.151661,151.399506,139.620300,256.059998,37.739998,262.269989,...,56.346661,106.908958,19.626289,77.655945,174.124283,166.672836,354.858337,208.831070,152.762863,52.974651
2021-03-19,118.743858,96.096710,116.934433,257.930908,45.196205,153.748001,137.789124,255.820007,37.342628,267.070007,...,55.693642,106.789993,19.462788,77.094696,181.821762,168.221405,358.318237,204.409317,151.895111,51.976727
2021-03-22,122.108551,98.401100,118.120804,259.319458,44.617134,155.543503,137.044907,251.229996,36.499447,272.339996,...,55.684311,105.472168,19.613203,76.832794,182.296814,172.159760,359.572815,200.016663,153.582932,51.063538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-22,174.626495,127.098175,136.696198,398.256805,54.307255,171.037003,160.389740,201.690002,43.525421,234.479996,...,64.116211,148.693207,17.418028,91.770309,213.228165,181.789764,489.385803,238.942108,201.762787,53.990273
2021-12-23,175.262817,128.099167,136.863373,397.881958,55.584614,171.068497,162.087708,204.220001,43.663036,235.410004,...,64.010391,148.626297,17.481291,92.539780,216.430405,182.246841,490.623779,241.647415,205.392654,54.221375
2021-12-29,178.344925,130.284042,138.859879,409.828949,55.790955,169.201004,161.732315,203.660004,43.869461,258.309998,...,65.482224,151.781219,17.319620,93.790192,223.197220,185.563080,500.725861,246.949814,207.725449,54.500618


In [24]:
index_difference = []

#Function runs for approx. 9 min for yfinance API
def filter_market(df):
    df_transposed = df.T
    mkt_lst = []
    
    for i in range(len(df_transposed)):
        # Populate market list
        mkt_lst.append(stocks_full_info[tickers_list[i]].info['market'])
    
    df_transposed['Market'] = mkt_lst
    
    # Drops the stocks in markets that are not in the U.S
    df_transposed.drop(df_transposed[df_transposed.Market != "us_market"].index, inplace=True)
    df_transposed = df_transposed.drop('Market', axis=1)
    return_df = df_transposed.T

    # Tracks the indices of the values that were dropped
    index_difference.append(df.difference(return_df.T, sort=False))

    return return_df

display(filter_market(all_prices))


KeyboardInterrupt: 

In [18]:
# Remove invalid stocks per assignment requirement (ones outside of the U.S & ones with average monthly volume of less than 200,000 shares)
def filter_volume(df):

    start = "2022-01-01"
    end = "2022-10-31"

    df_transposed = df.T
    volume_lst = []

    for i in range(len(df_transposed)):
        ticker_hist = stocks_full_info[tickers_list[i]].history(start=start, end=end)
        volume_hist = ticker_hist.loc[(ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        volume_lst.append(volume_hist['Volume'].mean()*30)

    # Drops the stocks with volumes less than average monthly volume of 200,000
    df_transposed["Volume"] = volume_lst

    df_transposed.drop(df_transposed[df_transposed.Volume < 200000].index, inplace=True)
    df_transposed = df_transposed.drop('Volume', axis=1)
    return_df = df_transposed.T

    # Tracks the indices of the values that were dropped
    index_difference.append(df.difference(return_df.T, sort=False))

    return return_df

display(filter_volume(all_prices))


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SO,SPG,T,TD.TO,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-16,124.265907,102.991287,114.960403,257.676666,45.678761,154.593002,140.325378,255.210007,36.480061,259.920013,...,57.139629,107.247574,19.613203,76.842155,174.705963,169.330246,347.556396,205.282028,155.261246,50.686966
2021-03-17,123.464310,97.601997,114.376953,258.410065,46.006893,156.786499,141.676697,263.589996,36.780502,264.149994,...,56.626530,108.208481,19.717846,76.935684,172.951233,171.442841,345.184448,205.863846,153.191956,51.298893
2021-03-18,119.278252,96.421921,114.853439,260.971985,46.151661,151.399506,139.620300,256.059998,37.739998,262.269989,...,56.346661,106.908958,19.626289,77.655945,174.124283,166.672836,354.858337,208.831070,152.762863,52.974651
2021-03-19,118.743858,96.096710,116.934433,257.930908,45.196205,153.748001,137.789124,255.820007,37.342628,267.070007,...,55.693642,106.789993,19.462788,77.094696,181.821762,168.221405,358.318237,204.409317,151.895111,51.976727
2021-03-22,122.108551,98.401100,118.120804,259.319458,44.617134,155.543503,137.044907,251.229996,36.499447,272.339996,...,55.684311,105.472168,19.613203,76.832794,182.296814,172.159760,359.572815,200.016663,153.582932,51.063538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-22,174.626495,127.098175,136.696198,398.256805,54.307255,171.037003,160.389740,201.690002,43.525421,234.479996,...,64.116211,148.693207,17.418028,91.770309,213.228165,181.789764,489.385803,238.942108,201.762787,53.990273
2021-12-23,175.262817,128.099167,136.863373,397.881958,55.584614,171.068497,162.087708,204.220001,43.663036,235.410004,...,64.010391,148.626297,17.481291,92.539780,216.430405,182.246841,490.623779,241.647415,205.392654,54.221375
2021-12-29,178.344925,130.284042,138.859879,409.828949,55.790955,169.201004,161.732315,203.660004,43.869461,258.309998,...,65.482224,151.781219,17.319620,93.790192,223.197220,185.563080,500.725861,246.949814,207.725449,54.500618


NameError: name 'index_difference' is not defined

In [ ]:

# all_prices = all_prices.loc[filter_market(all_prices)]
# all_prices = all_prices.loc[filter_volume(all_prices)]

# # Drops stocks with more than specified number of NaN values
# max_number_of_nans = 20
# all_prices = all_prices.loc[:, (all_prices.isnull().sum(axis=0) <= max_number_of_nans)]

In [27]:
# Reset previously defined variables
all_prices = filter_volume(all_prices)

for col in all_prices.columns:
     tickers_list[i] = col

for i in range(len(tickers_list)):
    stocks_full_info = stocks_full_info.pop(tickers_list[index_difference[i]])

tickers_num = len(tickers_list)


KeyboardInterrupt: 

In [ ]:
# Calculate BETA against S&P500
def stock_beta(df_column):

    index_ticker = yf.Ticker('SPY')
    index_prices = index_ticker.history(start=df_column.index[0], end=df_column.index[(len(df_column))])

    # Will need to double check this equation
    market_var= index_prices.var()
    beta = df_column.cov()/market_var
    
    return beta

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.